## paper ingestion
given a link, I want to ingest the title, abstract, doi, and the doi of the other papers it cites (as well as the papers those papers cite). how many levels down i go in the graph will depend on how stringent my filtering function is.  

currently using [this](https://api.semanticscholar.org/api-docs/graph#tag/Paper-Data/operation/get_graph_paper_bulk_search) api from semantic scholar and storing data locally as a json. in the futuer, will aim to use sqlite for persistance instead.



In [4]:

test_doi = '10.1371/journal.pone.0286404'
test_paper_id = 'c951ac9a54bba70c6e8337ab5815f3ac45434ec5'

def create_url(id, is_doi=False):
    endpoint = "https://api.semanticscholar.org/graph/v1/paper/"
    fields = '?fields=title,abstract,url,year,authors,references'
    if is_doi:
        return endpoint + "DOI:" + id + fields
    else: 
        return endpoint + id + fields

In [5]:
import requests

# calling the semantic scholar api
def api_call(api_endpoint):
    response = requests.get(api_endpoint)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(api_endpoint)
        return None


In [7]:
import time

# a lot of error handling, hasn't broken so far
def try_api_call(api_endpoint):
    try:
        response = requests.get(api_endpoint)
        if response.status_code == 429:
            print("Rate limit reached. Waiting before retrying...")
            time.sleep(20)  # Sleep for a minute, or an appropriate back-off time for your use case
            return api_call(api_endpoint)  # Retry the request
        response.raise_for_status()  # Will raise an HTTPError for other bad status codes
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {api_endpoint}")
    except Exception as err:
        print(f"An error occurred: {err} - {api_endpoint}")
    return None

In [26]:
import json
# pretty print json object
def pprint(data):
    print(json.dumps(data, indent=4, sort_keys=True))
    

In [31]:
# keywords
ml_keywords=[
    'machine learning', 
    'regression', 
    'neural network', 
    'model',
    'projecting',
    'forecast',
    'predict',
    'modeling',
    'classification', 
    'clustering', 
    'support vector machine', 
    'decision tree', 
    'random forest', 
    'learning', 
    'gradient boosting', 
    'data mining', 
    'natural language processing', 
    'computer vision', 
    'algorithm', 
    'optimization', 
]
conflict_keywords=[
    'armed', 
    'civil',
    'war', 
    'conflict',
    'insurgency', 
    'terrorism', 
    'extremism', 
    'revolution', 
    'violence', 
    'warfare', 
    'battle', 
    'combat', 
    'militia', 
    'security', 
    'peacekeeping', 
    'genocide', 
    'massacre', 
    'ceasefire', 
    'rebellion', 
    'humanitarian', 
    'occupation', 
]
all_keywords = ml_keywords + conflict_keywords

In [12]:
def keyword_check(string):
    string = string.lower()
    return any(keyword in string for keyword in all_keywords)

## used to create v0 of the graph
## generally checks for machine learning + conflicts related keywords (both must be present)
def double_keyword_check(string):
    string = string.lower()
    match1 = any(keyword.lower() in string for keyword in ml_keywords)
    match2 = any(keyword.lower() in string for keyword in conflict_keywords)
    return match1 and match2

def paper_is_relevant(json_blob):
    title = json_blob['title']
    if double_keyword_check(title):
        return True
    else:
        abstract = json_blob['abstract']
        return double_keyword_check(abstract)


In [14]:
# "database": will make this a sqlite later
# key: paperId, value: [api call data object, how many times hit]
papers = {}

In [15]:
def filter_refs(paper_refs):
    # first pass at filtering out references that don't pass the general keyword check
    filtered_references = [
        ref for ref in paper_refs
        if keyword_check(ref['title'])
    ]
    return filtered_references

# pprint(filter_refs(res))

In [16]:
# testing all the helpers thus far
    
def add_to_dict(res):
    if res:
        # there was a response
        id = res['paperId']
        if id in papers:
            print('paper already in database')
        else:
            refs = filter_refs(res.pop('references', None))
            papers[id] = [res, refs, 1]
        


In [17]:
def from_a_node(key):
    value = papers[key]
    # iterate through the refs
    for ref in value[1]:
        ref_id = ref['paperId']
        if ref_id in papers:
            # if the ref is already in the database, increment the hit count
            papers[ref_id][2] += 1
        else:  
            # if the ref is not in the database, add it
            url = create_url(ref_id)
            res = api_call(url)
            if res['abstract'] and double_keyword_check(res['abstract']):
                add_to_dict(res)
    # touched again
    papers[key][2] += 1

In [18]:
papers = {}

# add the first node
url = create_url(test_paper_id, False)
res = api_call(url)
add_to_dict(res)

# print(papers)
print(papers)

{'c951ac9a54bba70c6e8337ab5815f3ac45434ec5': [{'paperId': 'c951ac9a54bba70c6e8337ab5815f3ac45434ec5', 'url': 'https://www.semanticscholar.org/paper/c951ac9a54bba70c6e8337ab5815f3ac45434ec5', 'title': 'Modeling analysis of armed conflict risk in sub-Saharan Africa, 2000–2019', 'abstract': 'Sub-Saharan Africa has suffered frequent outbreaks of armed conflict since the end of the Cold War. Although several efforts have been made to understand the underlying causes of armed conflict and establish an early warning mechanism, there is still a lack of a comprehensive assessment approach to model the incidence risk of armed conflict well. Based on a large database of armed conflict events and related spatial datasets covering the period 2000–2019, this study uses a boosted regression tree (BRT) approach to model the spatiotemporal distribution of armed conflict risk in sub-Saharan Africa. Evaluation of accuracy indicates that the simulated models obtain high performance with an area under the 

In [89]:
# no need to run
from_a_node(test_paper_id)

# Define the filename where you want to store the data
filename = 'papers.json'

# Write the dictionary to a file as JSON
with open(filename, 'w') as file:
    json.dump(papers, file, indent=4)

print(f"Dictionary saved to {filename}")

Dictionary saved to papers.json


In [19]:
def save_graph_to_file(graph_data, file_name):
    with open(file_name, 'w') as file:
        json.dump(graph_data, file, indent=4, default=str)  # Using default=str to handle non-serializable data

In [32]:

## defaults to double keyword check (ML + conflicts) as filtering
def generate_reference_network(start_paper_id, max_depth=2, filter_func=double_keyword_check, res_name='graph.json'):
    graph = {}  # Initialize an empty graph, key: paper_id, value: paper_blob, list of references
    start_blob = try_api_call(create_url(start_paper_id))
    queue = [(start_paper_id, start_blob, 0)]  # Queue of (paper_id, current_depth)
    visited = set()  # Set of visited paper IDs to avoid duplicates
    save_interval = 1
    processed_papers = 0  

    while queue:
        current_paper_id, paper_info, current_depth = queue.pop(0)
        if current_paper_id not in visited and current_depth <= max_depth:
            print(f"Processing paper {current_paper_id} at depth {current_depth}")
            visited.add(current_paper_id)
    
            # Retrieve references and check if the paper meets the criteria
            references = filter_refs(paper_info.pop('references', None))
            print("------- the num of references are: ", len(references))
            relevant_refs = []
            for ref in references:
                ref_id = ref['paperId']
                if ref_id:
                    url = create_url(ref_id)
                    res = try_api_call(url)
                    if res and res['abstract'] and filter_func(res['abstract']):
                        relevant_refs.append(ref_id) 
                        queue.append((ref_id, res, current_depth + 1))
            
            graph[current_paper_id] = (paper_info, relevant_refs)
            print("------- the rel references are: ", relevant_refs)

            processed_papers += 1
            
            # Save the graph every three papers processed
            if processed_papers % save_interval == 0:
                save_graph_to_file(graph, res_name)
                print(f"Saved backup of graph at {processed_papers} papers.")
                
    return graph



In [ ]:
## took 23 min 40 sec to get to 129 papers
## rate is about 5 papers per minute

# This will return a graph dictionary with each paper ID as keys and 
# the tuple (paper_info, lists of relevant references) as values.
reference_network = generate_reference_network(test_paper_id)

## Random Forest Network
Trying this exclusively for papers about RF as a conflict modeling tactic


In [30]:
## RANDOM FOREST PAPERS
# first node: 
# 10.1371/journal.pone.0286404

rf_keywords = [
    'forest',
    'tree',
    'random',
    'decision tree',
    'regression trees',
    'boost',
    'gradient',
    'bagging',
    'boosting',
    'lightgbm',
    'cart',
    'adaboost',
]

## generally checks for keywords relating to random forest classification
def filter_trees(string):
    string = string.lower()
    match1 = any(keyword.lower() in string for keyword in rf_keywords)
    # for speed
    if not match1:
        return False
    match2 = any(keyword.lower() in string for keyword in conflict_keywords)
    return match1 and match2


In [22]:
## running the random forest tree generation 
# title: Comparing Random Forest with Logistic Regression for Predicting Class-Imbalanced Civil War Onset Data
# published: 2016

rf_doi = '10.1093/pan/mpv024'
# res = try_api_call(create_url(rf_doi, True))

rf_paper_id = '4b6555beef240120bacb699c7d2f7c8e806b5747'

In [35]:
## Actual generation
rf_network = generate_reference_network(rf_paper_id, max_depth=3, filter_func=filter_trees, res_name='rf_graph.json')

Processing paper 4b6555beef240120bacb699c7d2f7c8e806b5747 at depth 0
------- the num of references are:  47
------- the rel references are:  ['a4d4053fa12ac75164fe2df0b20a4d3883c292c3', 'da0a744bd257cc50a8778b4452b253bbe3c0b654', '3a8743b858ed557e3c722095ad4d4158e271e0bd', '17c7d2aada72fe59c8d282235b26089d526e44ee']
Saved backup of graph at 1 papers.
Processing paper a4d4053fa12ac75164fe2df0b20a4d3883c292c3 at depth 1
------- the num of references are:  27
------- the rel references are:  ['18c332b0bad296f8722677d2d63092bf02e0c210', '25badc676197a70aaf9911865eb03469e402ba57']
Saved backup of graph at 2 papers.
Processing paper da0a744bd257cc50a8778b4452b253bbe3c0b654 at depth 1
------- the num of references are:  12
------- the rel references are:  ['2d1e0a5d5309354d4948fa88b01232e59eee94dc']
Saved backup of graph at 3 papers.
Processing paper 3a8743b858ed557e3c722095ad4d4158e271e0bd at depth 1
------- the num of references are:  48
------- the rel references are:  ['34122df837465c1f20

In [34]:
# try_api_call(create_url("10.1007/978-3-319-55708-3_19", True))
